# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley 2018

Due Tuesday 2pm, Feb 20, 2018

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [78]:
# imports
import pyaudio
import wave
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import json
from urllib.request import urlopen
from word2number import w2n
import houndify

# import houndify and email params
# these files contain user specific information and thus need to be created by each use
import houndify_login as lg
import mail_params as mp

In [79]:
# create function to do record audio command and save as a wav file in format req'd for houndify
def rec_audio(fl_name):
    
    # initialize params for pyaudio
    chunk = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    
    # set up recording
    print('Welcome to Monty!\n')
    print('Please give me a request (type "ii" when done)\n')
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True,
        frames_per_buffer = chunk)
    all = []
    
    # record until keyboard is used to stop recording
    try:
        while True:
            data = stream.read(chunk)
            all.append(data)
    except KeyboardInterrupt:
        pass
    
    # close down recording
    stream.close()
    p.terminate()
    
    # write to .wav file
    data = b"".join(all)
    wf = wave.open(fl_name, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

In [80]:
# given a wav audio file, use houndify to return a list of all words spoken (in order) from audio file
def audio_to_text(wav_fl, ID, KEY):
    
    # open audio file
    audio = wave.open(wav_fl)
    
    # make sure audio file is in the correct format
    if audio.getsampwidth() != 2:
        print("wrong sample width (must be 16-bit)")
    if audio.getframerate() != 8000 and audio.getframerate() != 16000:
        print("unsupported sampling frequency (must be either 8 or 16 khz)")
    if audio.getnchannels() != 1:
        print("must be single channel (mono)")

    # connect to houndify and use it to perform audio to text conversion
    client = houndify.StreamingHoundClient(ID, KEY, "test_user")
    client.setSampleRate(audio.getframerate())
    client.start()
    while True:
        samples = audio.readframes(1024)
        if len(samples) == 0: break
        if client.fill(samples): break
    result = client.finish()
    
    # return a list of all words in audio file (in order)
    return result['AllResults'][0]['RawTranscription'].split(' ')

In [81]:
# function to take a raw request (from audio_to_text) and extract subject and body
def proc_for_email(request):
    
    # the subject of the email will be all the words in the list after 'subject' and before the 'and' before 'body' 
    subj = request[request.index('subject')+1:request.index('body')-1]
    
    # body of the email will be all the words after 'body'
    body = request[request.index('body')+1:]
    
    # return the subject and body of the message by joining each list subset into a string
    return ' '.join(subj), ' '.join(body)

In [9]:
# given sending address + password, to address, subject, and body, send email
# NB: this implementation requires one to set their Gmail account to accept "less secure apps"
#    https://myaccount.google.com/u/4/lesssecureapps
def send_email(sender, pwd, to, subject, body):
    
    # setup message attributes
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = to # one sender only
    #msg["Date"] = formatdate(localtime=True)
    msg["Subject"] = subject
    msg.attach(MIMEText(body))
    
    # send message
    mailServer = smtplib.SMTP("smtp.gmail.com", 587)
    mailServer.ehlo()
    mailServer.starttls()
    mailServer.login(sender, pwd)
    mailServer.sendmail(sender, to, msg.as_string())
    mailServer.close()

In [82]:
# obtain a random joke from the internet chuck norris database
def pull_joke(url = 'http://api.icndb.com/jokes/random'):
    
    # query the internet database and store result as json
    result = json.loads(urlopen(url).read().decode("UTF-8"))
    
    # print the joke
    print(result['value']['joke'])

In [83]:
# take words containing a mathematical expression and evaluate the expression
def math_from_words(request):
    
    # while I could imagine writing a function with a large dictionary to parse numbers as words into integers
    #    or floats, this would be somewhat cumbersome - especially when there is a nice python package that does it.
    #    I will use the 
    
    # create dictionary of mathematical symbols to interpret
    symbols = {'plus': '+', 'minus': '-', 'times': '*', 'divided': '/'}
    
    # create a new list to hold items from the request that are part of the mathematical expression
    expr_list = []
    
    # create an empty string to hold sequential words that correspond to one number
    expr = ''
    
    # iterate through the request and extract items that are part of the expression
    for idx, word in enumerate(request):
        
        # check to see if the word is a symbol, if so add it
        if word in symbols.keys():
            
            # if the word is symbol write the current number expression to the list and re-initialize an empty one
            # the rational here is that symbols separate sequences of words that represent a single number
            expr_list.append(expr)
            expr = ''
            
            # if 'divided' is the word, require that it be followed by 'by' to count as symbol
            if word != 'divided':
                expr_list.append(symbols[word])
            elif word == 'divided' and request[idx +1] == 'by':
                expr_list.append(symbols[word])
            
        # if the word isn't a symbol, check to see if it is the number 4, but spelled as 'for' - if so include it
        elif word == 'for':
            expr += ' four'
            
        # if the word is 'point', set it up to be interpreted as a decimal point
        elif word == 'point':
            expr += ' point'
        
        # if the word doesn't meet the above conditions it should either be a number or an irrelevant word
        #     it will be an irrelevant word if the function throws a value error - catch and handle this
        #     if there is not a value error, then it is a number - include it
        else:
            try:
                tmp = str(w2n.word_to_num(word))
                expr += word
            except ValueError:
                pass
    
    # add the last sequence of numbers to the expression list - the conditions make it possible for there to be
    #     a non-empty sequence stored when iteration ends
    expr_list.append(expr)
    
    # initialize an expression to evaluate
    expr_to_eval = ''
    
    # iterate through the list holding the mathematical expression
    for item in expr_list:
        
        # if the current item is a symbol, add it to the expression to evaluate
        if item in symbols.values():
            expr_to_eval += item
        
        # otherwise the current item is a sequence corresponding to one number - convert this from words to a number
        else:
            expr_to_eval += str(w2n.word_to_num(item))
    
    # evaluate the mathematical expression and print the result
    print(eval(expr_to_eval))

In [84]:
# function that completely controls and runs monty
def monty(ID, KEY, fl_name = 'tmp_audio/tmp.wav', print_req = False):
    
    # record audio and store it as a wav file
    rec_audio(fl_name)
    
    # get a list of words (in order) from the audio file
    request = audio_to_text(fl_name, ID, KEY)
    
    # optionally print the text obtained from audio
    if print_req:
        print(request)
    
    # determine what to do
    
    # if no words were registered, let the user know
    if len(request) == 1:
        print("Sorry, I didn't get that. Please try again.")
    
    # detect an email request by having the words 'email', 'subject', and 'body' all appear in request
    elif set(('email','subject','body')).issubset(set(request)):
        subject, body = proc_for_email(request)
        send_email(mp.sender, mp.pwd, mp.sender, subject, body)
        print('email sent!')
        
    # detect a joke request by having the words 'tell' and 'joke' in request and having request not be an email
    elif set(('tell','joke')).issubset(set(request)):
        pull_joke()
        
    # if it isn't an email or joke request, assume it is a math request
    else:
        math_from_words(request)

In [85]:
monty(lg.ID, lg.KEY)

Welcome to Monty!

Please give me a request (type "ii" when done)

Sorry, I didn't get that. Please try again.


# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.